In [118]:
import torch
from torch import Tensor
from torch.autograd import Variable
from torch import nn
from torch.nn import functional as F

import dlc_bci as bci
from helpers import *

torch.manual_seed(1)

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Importing the data

In [119]:
# Subset of data sampled at 100Hz
train_input , train_target = bci.load(root = './data_bci')
print("Train input :", str(type(train_input)), train_input.size()) 
print("Train target :", str(type(train_target)), train_target.size())
test_input , test_target = bci.load(root = './data_bci', train = False)
print("Test input :", str(type(test_input)), test_input.size()) 
print("Test target :", str(type(test_target)), test_target.size())

Train input : <class 'torch.FloatTensor'> torch.Size([316, 28, 50])
Train target : <class 'torch.LongTensor'> torch.Size([316])
Test input : <class 'torch.FloatTensor'> torch.Size([100, 28, 50])
Test target : <class 'torch.LongTensor'> torch.Size([100])


In [81]:
# Full data sampled at 1Khz
train_input , train_target = bci.load(root = './data_bci', one_khz = True)
print("Train input :", str(type(train_input)), train_input.size()) 
print("Train target :", str(type(train_target)), train_target.size())
test_input , test_target = bci.load(root = './data_bci', train = False, one_khz = True)
print("Test input :", str(type(test_input)), test_input.size()) 
print("Test target :", str(type(test_target)), test_target.size())

<class 'torch.FloatTensor'> torch.Size([316, 28, 500])
<class 'torch.LongTensor'> torch.Size([316])
<class 'torch.FloatTensor'> torch.Size([100, 28, 500])
<class 'torch.LongTensor'> torch.Size([100])


In [82]:
train_input[0]


  30.8000   33.1000   37.9000  ...    76.9000   75.9000   72.3000
 -22.6000  -19.2000  -11.8000  ...    26.8000   22.9000   16.4000
  11.2000   16.7000   26.4000  ...    64.1000   62.0000   55.9000
             ...                ⋱                ...             
   0.5000    0.5000    0.4000  ...    45.1000   46.2000   46.6000
  11.0000   10.6000    9.8000  ...    41.7000   41.5000   40.9000
  -9.9000  -10.0000  -10.3000  ...    32.9000   33.8000   32.8000
[torch.FloatTensor of size 28x500]

In [83]:
train_target


 0
 1
 0
 1
 0
 1
 0
 1
 0
 0
 1
 1
 0
 0
 1
 1
 1
 0
 0
 1
 0
 0
 1
 0
 1
 0
 0
 0
 0
 0
 0
 0
 1
 1
 1
 1
 1
 1
 0
 1
 0
 1
 0
 0
 1
 0
 1
 1
 0
 1
 1
 0
 0
 1
 0
 0
 1
 0
 1
 0
 1
 0
 0
 1
 0
 0
 0
 0
 0
 1
 0
 1
 0
 0
 0
 0
 0
 1
 1
 1
 0
 1
 0
 1
 1
 1
 1
 1
 1
 1
 1
 1
 0
 0
 0
 0
 1
 1
 1
 0
 1
 1
 1
 0
 1
 0
 1
 1
 1
 0
 1
 0
 1
 0
 0
 0
 1
 0
 0
 1
 1
 1
 1
 0
 1
 1
 0
 0
 1
 1
 1
 1
 1
 0
 0
 0
 0
 1
 1
 0
 1
 1
 1
 1
 1
 1
 1
 0
 0
 0
 1
 1
 0
 0
 1
 1
 0
 0
 0
 0
 0
 1
 1
 1
 1
 1
 1
 0
 1
 0
 1
 1
 0
 1
 0
 0
 1
 1
 1
 0
 0
 1
 0
 0
 0
 0
 0
 1
 1
 0
 0
 0
 0
 1
 0
 0
 0
 1
 0
 0
 1
 0
 0
 0
 1
 1
 1
 0
 0
 0
 1
 1
 1
 0
 0
 0
 1
 0
 1
 1
 1
 1
 1
 1
 1
 0
 1
 0
 0
 0
 1
 0
 1
 1
 1
 0
 1
 1
 0
 0
 0
 1
 1
 0
 1
 1
 0
 1
 1
 1
 1
 1
 0
 1
 0
 1
 0
 0
 0
 0
 0
 0
 1
 1
 1
 0
 1
 1
 1
 0
 1
 0
 0
 1
 1
 0
 0
 1
 0
 0
 1
 0
 0
 0
 1
 0
 0
 1
 1
 0
 0
 1
 1
 1
 0
 1
 0
 1
 1
 1
 0
 1
 1
 0
 1
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
[torch.LongTensor of size 316]

### Preprocessing the data

In [120]:
# normalize mean to 0
train_input.sub_(train_input.mean())
test_input.sub_(test_input.mean())

# normalize variance to 1
train_input.div_(train_input.std())
test_input.div_(test_input.std())

print('done')

done


## Linear Regression

In [156]:
from sklearn.linear_model import LinearRegression

In [157]:
linear_reg = LinearRegression()

In [158]:
linear_reg.fit(train_input.view(train_input.size(0),-1),train_target)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [174]:
linear_pred_train = discrete_predictions(torch.FloatTensor(linear_reg.predict(train_input.view(train_input.size(0),-1))))

In [175]:
print(100*((torch.LongTensor(linear_pred_train) - train_target).abs().sum()/train_input.size(0)),"% train error")

0.0 % train error


In [176]:
linear_pred_test = discrete_predictions(torch.FloatTensor(linear_reg.predict(test_input.view(test_input.size(0),-1))))

In [177]:
print(100*((torch.LongTensor(linear_pred_test) - test_target).abs().sum()/test_input.size(0)),"% test error")

27.0 % test error


## Logistic Regression

In [149]:
from sklearn.linear_model import LogisticRegression

In [150]:
logistic_reg = LogisticRegression()

In [151]:
logistic_reg.fit(train_input.view(train_input.size(0),-1),train_target)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [152]:
logistic_pred_train = logistic_reg.predict(train_input.view(train_input.size(0),-1))

In [153]:
print(100*((torch.LongTensor(logistic_pred_train) - train_target).abs().sum()/train_input.size(0)),"% train error")

0.0 % train error


In [154]:
logistic_pred_test = logistic_reg.predict(test_input.view(test_input.size(0),-1))

In [155]:
print(100*((torch.LongTensor(logistic_pred_test) - test_target).abs().sum()/test_input.size(0)),"% test error")

28.000000000000004 % test error


## Neural Networks

In [10]:
#convert targets to one hot labels
#train_target = convert_to_one_hot_labels(train_input, train_target)
#test_target = convert_to_one_hot_labels(test_input, test_target)

### Basic MLP

In [97]:
MLP_big = nn.Sequential(
        nn.Linear(14000, 1400),
        nn.Tanh(),
        nn.Linear(1400, 280),
        nn.Tanh(),
        nn.Linear(280, 28),
        nn.Tanh(),
        nn.Linear(28, 2)
    )

In [129]:
MLP = nn.Sequential(
        nn.Linear(1400, 280),
        nn.Tanh(),
        nn.Linear(280, 140),
        nn.Tanh(),
        nn.Linear(140, 28),
        nn.Tanh(),
        nn.Linear(28, 2)
    )

In [130]:
train_model(MLP, Variable(train_input.view(train_input.size(0),-1)), Variable(train_target), 4)

Epoch 0 loss = 55.861808478832245
Epoch 1 loss = 54.59076404571533
Epoch 2 loss = 53.797380566596985
Epoch 3 loss = 53.191094636917114
Epoch 4 loss = 52.66807159781456
Epoch 5 loss = 52.18098449707031
Epoch 6 loss = 51.69751679897308
Epoch 7 loss = 51.194580405950546
Epoch 8 loss = 50.65688365697861
Epoch 9 loss = 50.074628949165344
Epoch 10 loss = 49.44200122356415
Epoch 11 loss = 48.756370306015015
Epoch 12 loss = 48.019282549619675
Epoch 13 loss = 47.23957505822182
Epoch 14 loss = 46.418880090117455
Epoch 15 loss = 45.549498587846756
Epoch 16 loss = 44.65137520432472
Epoch 17 loss = 43.72418200969696
Epoch 18 loss = 42.76705865561962
Epoch 19 loss = 41.78091523051262
Epoch 20 loss = 40.76593078672886
Epoch 21 loss = 39.72083881497383
Epoch 22 loss = 38.644645400345325
Epoch 23 loss = 37.53784937411547
Epoch 24 loss = 36.39436551183462
Epoch 25 loss = 35.19167825952172
Epoch 26 loss = 33.98767836764455
Epoch 27 loss = 32.73168759420514
Epoch 28 loss = 31.445499220862985
Epoch 29 loss

In [131]:
train_error_mlp = compute_nb_errors(MLP, Variable(test_input.view(test_input.size(0),-1)), Variable(train_target), 4)
print(100*(train_error_mlp/train_input.size(0)),'% training error')

16.455696202531644 % training error


In [132]:
test_error_mlp = compute_nb_errors(MLP, Variable(test_input.view(test_input.size(0),-1)), Variable(test_target), 4)
print(100*(test_error_mlp/test_input.size(0)),'% test error')

28.999999999999996 % test error


### Basic CNN

In [179]:
class Basic_CNN(nn.Module):
    def __init__(self):
        super(Basic_CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5)
        self.fc1 = nn.Linear(640, 2)

    def forward(self, x):
        x = F.tanh(F.max_pool2d(self.conv1(x), kernel_size=3, stride=3))
        x = F.tanh(F.max_pool2d(self.conv2(x), kernel_size=2, stride=2))
        x = F.tanh(self.fc1(x.view(-1, 640)))
        return x

In [180]:
cnn = Basic_CNN()

In [181]:
train_model(cnn, Variable(train_input.view(-1, 1, 28, 50)), Variable(train_target), 4)

Epoch 0 loss = 56.96408760547638
Epoch 1 loss = 55.346612334251404
Epoch 2 loss = 54.624076545238495
Epoch 3 loss = 54.15493232011795
Epoch 4 loss = 53.71287339925766
Epoch 5 loss = 53.25642216205597
Epoch 6 loss = 52.768557608127594
Epoch 7 loss = 52.23355823755264
Epoch 8 loss = 51.66652154922485
Epoch 9 loss = 51.05329883098602
Epoch 10 loss = 50.3758901655674
Epoch 11 loss = 49.651860892772675
Epoch 12 loss = 48.8626429438591
Epoch 13 loss = 48.01820057630539
Epoch 14 loss = 47.1332391500473
Epoch 15 loss = 46.1921281516552
Epoch 16 loss = 45.22527086734772
Epoch 17 loss = 44.226421266794205
Epoch 18 loss = 43.21801918745041
Epoch 19 loss = 42.19562944769859
Epoch 20 loss = 41.18148720264435
Epoch 21 loss = 40.16077479720116
Epoch 22 loss = 39.14418172836304
Epoch 23 loss = 38.14010426402092
Epoch 24 loss = 37.14081862568855
Epoch 25 loss = 36.13838002085686
Epoch 26 loss = 35.16330511868
Epoch 27 loss = 34.177098259329796
Epoch 28 loss = 33.21918261051178
Epoch 29 loss = 32.275271

In [182]:
train_error_cnn = compute_nb_errors(cnn, Variable(train_input.view(-1, 1, 28, 50)), Variable(train_target), 4)
print(100*(train_error_cnn/train_input.size(0)),'% training error')

0.0 % training error


In [183]:
test_error_cnn = compute_nb_errors(cnn, Variable(test_input.view(-1, 1, 28, 50)), Variable(test_target), 4)
print(100*(test_error_cnn/test_input.size(0)),'% test error')

23.0 % test error


### CNN w 1D filters

In [186]:
class CNN_1D_conv(nn.Module):
    def __init__(self):
        super(CNN_1D_conv, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=(1,5))
        self.conv2 = nn.Conv2d(16, 32, kernel_size=(1,3))
        self.conv3 = nn.Conv2d(32, 64, kernel_size=(1,5))
        self.fc1 = nn.Linear(896*2, 128)
        self.fc2 = nn.Linear(128, 2)

    def forward(self, x):
        x = F.tanh(F.max_pool2d(self.conv1(x), kernel_size=(1,2), stride=(1,2)))
        x = F.tanh(F.max_pool2d(self.conv2(x), kernel_size=(1,3), stride=(1,3)))
        x = F.tanh(F.max_pool2d(self.conv3(x), kernel_size=(1,3), stride=(1,3)))
        x = F.tanh(self.fc1(x.view(-1, 896*2)))
        x = self.fc2(x)
        return x

In [187]:
cnn2 = CNN_1D_conv()

In [188]:
train_model(cnn2, Variable(train_input.view(-1, 1, 28, 50)), Variable(train_target), 4)

Epoch 0 loss = 55.29474902153015
Epoch 1 loss = 55.05597680807114
Epoch 2 loss = 54.855808675289154
Epoch 3 loss = 54.682802975177765
Epoch 4 loss = 54.524170219898224
Epoch 5 loss = 54.372238516807556
Epoch 6 loss = 54.2225626707077
Epoch 7 loss = 54.07223069667816
Epoch 8 loss = 53.92045372724533
Epoch 9 loss = 53.7664595246315
Epoch 10 loss = 53.6088582277298
Epoch 11 loss = 53.44771748781204
Epoch 12 loss = 53.28130882978439
Epoch 13 loss = 53.10787773132324
Epoch 14 loss = 52.924775540828705
Epoch 15 loss = 52.73032104969025
Epoch 16 loss = 52.52050358057022
Epoch 17 loss = 52.2923142015934
Epoch 18 loss = 52.04249811172485
Epoch 19 loss = 51.76665839552879
Epoch 20 loss = 51.46092280745506
Epoch 21 loss = 51.12171161174774
Epoch 22 loss = 50.74609687924385
Epoch 23 loss = 50.327125549316406
Epoch 24 loss = 49.860064417123795
Epoch 25 loss = 49.33736914396286
Epoch 26 loss = 48.7524134516716
Epoch 27 loss = 48.09661191701889
Epoch 28 loss = 47.363799035549164
Epoch 29 loss = 46.54

In [189]:
train_error_cnn2 = compute_nb_errors(cnn2, Variable(train_input.view(-1, 1, 28, 50)), Variable(train_target), 4)
print(100*(train_error_cnn2/train_input.size(0)),'% training error')

0.0 % training error


In [190]:
test_error_cnn2 = compute_nb_errors(cnn2, Variable(test_input.view(-1, 1, 28, 50)), Variable(test_target), 4)
print(100*(test_error_cnn2/test_input.size(0)),'% test error')

23.0 % test error


### previous + dropout

In [220]:
class CNN_dropout(nn.Module):
    def __init__(self):
        super(CNN_dropout, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=(1,5))
        self.conv2 = nn.Conv2d(16, 32, kernel_size=(1,3))
        self.conv3 = nn.Conv2d(32, 64, kernel_size=(1,5))
        self.fc1 = nn.Linear(896*2, 128)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(128, 2)

    def forward(self, x):
        x = F.tanh(F.max_pool2d(self.conv1(x), kernel_size=(1,2), stride=(1,2)))
        x = F.tanh(F.max_pool2d(self.conv2(x), kernel_size=(1,3), stride=(1,3)))
        x = F.tanh(F.max_pool2d(self.conv3(x), kernel_size=(1,3), stride=(1,3)))
        x = F.tanh(self.fc1(x.view(-1, 896*2))) 
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [221]:
cnn3 = CNN_dropout()

In [222]:
train_model(cnn3, Variable(train_input.view(-1, 1, 28, 50)), Variable(train_target), 4)

Epoch 0 loss = 56.019174098968506
Epoch 1 loss = 55.4520201086998
Epoch 2 loss = 54.7875514626503
Epoch 3 loss = 55.15742391347885
Epoch 4 loss = 54.50948143005371
Epoch 5 loss = 54.30815577507019
Epoch 6 loss = 53.86229258775711
Epoch 7 loss = 53.462171137332916
Epoch 8 loss = 53.096127450466156
Epoch 9 loss = 53.54497683048248
Epoch 10 loss = 53.712387442588806
Epoch 11 loss = 53.26724320650101
Epoch 12 loss = 52.391912400722504
Epoch 13 loss = 52.46445178985596
Epoch 14 loss = 52.34692305326462
Epoch 15 loss = 51.58625024557114
Epoch 16 loss = 52.29400381445885
Epoch 17 loss = 51.25133517384529
Epoch 18 loss = 51.64257895946503
Epoch 19 loss = 50.183215379714966
Epoch 20 loss = 49.77911111712456
Epoch 21 loss = 49.276940166950226
Epoch 22 loss = 48.689226537942886
Epoch 23 loss = 48.142345398664474
Epoch 24 loss = 46.38583126664162
Epoch 25 loss = 46.18621206283569
Epoch 26 loss = 44.67905259132385
Epoch 27 loss = 44.67619952559471
Epoch 28 loss = 43.307352751493454
Epoch 29 loss = 

In [223]:
train_error_cnn3 = compute_nb_errors(cnn3, Variable(train_input.view(-1, 1, 28, 50)), Variable(train_target), 4)
print(100*(train_error_cnn3/train_input.size(0)),'% training error')

0.0 % training error


In [224]:
test_error_cnn3 = compute_nb_errors(cnn3, Variable(test_input.view(-1, 1, 28, 50)), Variable(test_target), 4)
print(100*(test_error_cnn3/test_input.size(0)),'% test error')

26.0 % test error
